# Merge Distribution of Household Income data into a single data file

## Imports

In [1]:
import pandas as pd
import os
import re
from functools import reduce

## Collect list of files from data directory

In [2]:
files = os.listdir('data')
files

['table_01_demographics_1979_2014.csv',
 'table_02_income_group_minimums_1979_2014.csv',
 'table_03_average_household_income_1979_2014.csv',
 'table_04_median_household_income_1979_2014.csv',
 'table_05_components_inc_before_transfers_taxes_1979_2014.csv',
 'table_06_components_means_tested_transfers_1979_2014.csv',
 'table_07_components_federal_taxes_1979_2014.csv',
 'table_08_means_tested_transfer_rates_1979_2014.csv',
 'table_09_federal_tax_rates_1979_2014.csv',
 'table_10_household_income_shares_1979_2014.csv',
 'table_11_means_tested_transfer_shares_1979_2014.csv',
 'table_12_federal_tax_shares_1979_2014.csv']

In [3]:
df_list = []
for file in files:
    if not re.search('minimum|median', file):
        df = pd.read_csv('data\\' + file)
        
        # Add "_rates" to end of column names for rates tables
        if re.search('rates', file):
            for column in df.columns:
                if column not in ['household_type', 'income_group', 'year']:
                    df.rename(columns = {column : column + '_rate'}, inplace=True)
        
        # Add "share_of_" to front of column names for shares tables
        if re.search('shares', file):
            for column in df.columns:
                if column not in ['household_type', 'income_group', 'year']:
                    df.rename(columns = {column : 'share_of_' + column}, inplace=True)
        
        df_list.append(df)

In [4]:
all_data = reduce(lambda df1, df2: pd.merge(df1, df2, on = ['household_type', 'income_group', 'year']), df_list)

In [5]:
all_data.head()

,household_type,income_group,year,num_households,num_children,num_adults,num_elderly,num_people,market_income_x,social_insurance_benefits_x,...,share_of_all_means_tested_transfers,share_of_medicaid,share_of_snap,share_of_ssi,share_of_other_transfers,share_of_all_federal_taxes,share_of_individual_income_tax,share_of_payroll_taxes,share_of_corporate_income_tax,share_of_excise_taxes
0,all_households,all_quintiles,1979,81.1,64.8,132.9,24.2,221.9,60300.0,4500.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,all_households,all_quintiles,1980,82.6,64.5,135.2,24.7,224.5,58100.0,4900.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,all_households,all_quintiles,1981,83.8,63.9,136.8,25.3,226.0,58100.0,5100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,all_households,all_quintiles,1982,84.3,63.2,138.5,25.8,227.6,57700.0,5600.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,all_households,all_quintiles,1983,85.8,63.3,140.0,26.1,229.4,58300.0,5700.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [6]:
all_data.columns

Index(['household_type', 'income_group', 'year', 'num_households',
       'num_children', 'num_adults', 'num_elderly', 'num_people',
       'market_income_x', 'social_insurance_benefits_x',
       'inc_before_transfers_taxes_x', 'means_tested_transfers_x',
       'federal_taxes_x', 'inc_after_transfers_taxes',
       'inc_before_transfers_taxes_y', 'market_income_y', 'wages',
       'employee_contrib_deferred_comp', 'employer_contrib_health_ins',
       'employer_share_payroll_taxes', 'federal_unemployment_tax',
       'corp_tax_borne_by_labor', 'business_income', 'capital_gains',
       'tax_exempt_interest', 'taxable_interest', 'positive_rental_income',
       'dividends', 'corp_tax_borne_by_capital', 'other_market_income',
       'social_insurance_benefits_y', 'social_security', 'medicare',
       'unemployment_insurance', 'workers_compensation',
       'means_tested_transfers_y', 'medicaid', 'snap', 'ssi',
       'other_transfers', 'federal_taxes_y', 'individual_income_tax',
      

In [7]:
# Clean up duplicate columns
for column in all_data.columns:
    if column.endswith('_x'):
        all_data.rename(columns={column : column[0:-2]}, inplace=True)
    if column.endswith('_y'):
        all_data.drop(column, axis=1, inplace=True)

In [8]:
print ("all_data now has ", len(all_data.columns), "columns")
print (all_data.columns)

all_data now has  62 columns
Index(['household_type', 'income_group', 'year', 'num_households',
       'num_children', 'num_adults', 'num_elderly', 'num_people',
       'market_income', 'social_insurance_benefits',
       'inc_before_transfers_taxes', 'means_tested_transfers', 'federal_taxes',
       'inc_after_transfers_taxes', 'wages', 'employee_contrib_deferred_comp',
       'employer_contrib_health_ins', 'employer_share_payroll_taxes',
       'federal_unemployment_tax', 'corp_tax_borne_by_labor',
       'business_income', 'capital_gains', 'tax_exempt_interest',
       'taxable_interest', 'positive_rental_income', 'dividends',
       'corp_tax_borne_by_capital', 'other_market_income', 'social_security',
       'medicare', 'unemployment_insurance', 'workers_compensation',
       'medicaid', 'snap', 'ssi', 'other_transfers', 'individual_income_tax',
       'payroll_taxes', 'corporate_income_tax', 'excise_taxes',
       'means_tested_transfers_rate', 'medicaid_rate', 'snap_rate', 'ssi_

## Merge in medians
Only merging on `['household_type'] == 'all_households'` and `['income_group'] == 'all_quintiles'`

In [9]:
medians = pd.read_csv('data\\table_04_median_household_income_1979_2014.csv')
medians.head()

,year,inc_before_transfers_taxes,adj_inc_before_transfers_taxes,inc_after_transfers_taxes,adj_inc_after_transfers_taxes
0,1979,61600,33700,50600,27600
1,1980,59500,32800,49100,26700
2,1981,59400,33000,48700,26700
3,1982,58500,32600,48700,26800
4,1983,58000,32400,48300,26800


In [10]:
for column in medians.columns:
    if not column == 'year':
        medians.rename(columns={column : 'median_' + column}, inplace=True)
medians.head()

,year,median_inc_before_transfers_taxes,median_adj_inc_before_transfers_taxes,median_inc_after_transfers_taxes,median_adj_inc_after_transfers_taxes
0,1979,61600,33700,50600,27600
1,1980,59500,32800,49100,26700
2,1981,59400,33000,48700,26700
3,1982,58500,32600,48700,26800
4,1983,58000,32400,48300,26800


In [11]:
medians['household_type'] = 'all_households'
medians['income_group'] = 'all_quintiles'

In [12]:
all_data = pd.merge(all_data, medians, on=['household_type', 'income_group', 'year'], how='left')

In [13]:
print ("all_data now has ", len(all_data.columns), "columns")
print (all_data.columns)

all_data now has  66 columns
Index(['household_type', 'income_group', 'year', 'num_households',
       'num_children', 'num_adults', 'num_elderly', 'num_people',
       'market_income', 'social_insurance_benefits',
       'inc_before_transfers_taxes', 'means_tested_transfers', 'federal_taxes',
       'inc_after_transfers_taxes', 'wages', 'employee_contrib_deferred_comp',
       'employer_contrib_health_ins', 'employer_share_payroll_taxes',
       'federal_unemployment_tax', 'corp_tax_borne_by_labor',
       'business_income', 'capital_gains', 'tax_exempt_interest',
       'taxable_interest', 'positive_rental_income', 'dividends',
       'corp_tax_borne_by_capital', 'other_market_income', 'social_security',
       'medicare', 'unemployment_insurance', 'workers_compensation',
       'medicaid', 'snap', 'ssi', 'other_transfers', 'individual_income_tax',
       'payroll_taxes', 'corporate_income_tax', 'excise_taxes',
       'means_tested_transfers_rate', 'medicaid_rate', 'snap_rate', 'ssi_

## Merge on minimums


In [14]:
# Read in data
minimums = pd.read_csv('data\\table_02_income_group_minimums_1979_2014.csv')
minimums.head()

,household_size,year,lowest_quintile,second_quintile,middle_quintile,fourth_quintile,highest_quintile,percentiles_81_90,percentiles_91_95,percentiles_96_99,top_1_percent
0,1,1979,0,17300,28400,39300,54800,54800,70100,88500,169900
1,1,1980,0,16600,27500,38200,53700,53700,69200,86700,167000
2,1,1981,0,16500,27600,38700,54300,54300,69600,87800,165600
3,1,1982,0,16100,27100,38400,54500,54500,70100,87500,166100
4,1,1983,0,15600,26900,38500,55100,55100,71300,90100,173600


In [15]:
# Melt dataframe so there's a single column for the values (taken from all columns except 'household_size' and 'year')
val_vars = minimums.columns.difference(['household_size', 'year'])

melted_minimums = pd.melt(minimums, 
                          id_vars = ['household_size', 'year'], 
                          value_vars = list(val_vars))
melted_minimums

,household_size,year,variable,value
0,1,1979,fourth_quintile,39300
1,1,1980,fourth_quintile,38200
2,1,1981,fourth_quintile,38700
3,1,1982,fourth_quintile,38400
4,1,1983,fourth_quintile,38500
5,1,1984,fourth_quintile,40600
6,1,1985,fourth_quintile,40800
7,1,1986,fourth_quintile,41900
8,1,1987,fourth_quintile,42500
9,1,1988,fourth_quintile,43100


In [16]:
# Next, pivot the dataframe to get columns by household size
pivoted_minimums = melted_minimums.pivot_table(index=['year', 'variable'], columns='household_size', values='value')

In [17]:
pivoted_minimums.head(18)

household_size               1       2       3       4
year variable                                         
1979 fourth_quintile     39300   55600   68000   78600
     highest_quintile    54800   77500   94900  109600
     lowest_quintile         0       0       0       0
     middle_quintile     28400   40200   49300   56900
     percentiles_81_90   54800   77500   94900  109600
     percentiles_91_95   70100   99100  121400  140100
     percentiles_96_99   88500  125100  153200  176900
     second_quintile     17300   24500   30000   34700
     top_1_percent      169900  240300  294300  339800
1980 fourth_quintile     38200   54100   66200   76500
     highest_quintile    53700   76000   93100  107500
     lowest_quintile         0       0       0       0
     middle_quintile     27500   38900   47700   55100
     percentiles_81_90   53700   76000   93100  107500
     percentiles_91_95   69200   97900  119900  138400
     percentiles_96_99   86700  122500  150100  173300
     second_quintile     16600   23500   28800   33200
     top_1_percent      167000  236200  289300  334100

In [18]:
pivoted_minimums.tail(18)

household_size               1       2       3       4
year variable                                         
2013 fourth_quintile     54500   77100   94500  109100
     highest_quintile    83000  117300  143700  165900
     lowest_quintile         0       0       0       0
     middle_quintile     35900   50800   62200   71800
     percentiles_81_90   83000  117300  143700  165900
     percentiles_91_95  115000  162600  199100  229900
     percentiles_96_99  157100  222100  272000  314100
     second_quintile     21100   29900   36600   42300
     top_1_percent      357100  505000  618500  714200
2014 fourth_quintile     55000   77700   95200  109900
     highest_quintile    84700  119800  146700  169400
     lowest_quintile         0       0       0       0
     middle_quintile     36300   51400   62900   72600
     percentiles_81_90   84700  119800  146700  169400
     percentiles_91_95  117600  166400  203700  235300
     percentiles_96_99  162000  229100  280600  324000
     second_quintile     21100   29900   36600   42200
     top_1_percent      374400  529500  648500  748800

In [19]:
# Create meaningful variable names
for column in pivoted_minimums.columns:
    if not column == 'year':
        pivoted_minimums.rename(columns={column : 'minimum_income_' + pivoted_minimums.columns.name + '_' + str(column)}, inplace=True)

# print (pivoted_minimums.columns.name)        
# pivoted_minimums.columns.name = ''

In [20]:
pivoted_minimums.head(18)

household_size          minimum_income_household_size_1  \
year variable                                             
1979 fourth_quintile                              39300   
     highest_quintile                             54800   
     lowest_quintile                                  0   
     middle_quintile                              28400   
     percentiles_81_90                            54800   
     percentiles_91_95                            70100   
     percentiles_96_99                            88500   
     second_quintile                              17300   
     top_1_percent                               169900   
1980 fourth_quintile                              38200   
     highest_quintile                             53700   
     lowest_quintile                                  0   
     middle_quintile                              27500   
     percentiles_81_90                            53700   
     percentiles_91_95                            69200   
     percentiles_96_99                            86700   
     second_quintile                              16600   
     top_1_percent                               167000   

household_size          minimum_income_household_size_2  \
year variable                                             
1979 fourth_quintile                              55600   
     highest_quintile                             77500   
     lowest_quintile                                  0   
     middle_quintile                              40200   
     percentiles_81_90                            77500   
     percentiles_91_95                            99100   
     percentiles_96_99                           125100   
     second_quintile                              24500   
     top_1_percent                               240300   
1980 fourth_quintile                              54100   
     highest_quintile                             76000   
     lowest_quintile                                  0   
     middle_quintile                              38900   
     percentiles_81_90                            76000   
     percentiles_91_95                            97900   
     percentiles_96_99                           122500   
     second_quintile                              23500   
     top_1_percent                               236200   

household_size          minimum_income_household_size_3  \
year variable                                             
1979 fourth_quintile                              68000   
     highest_quintile                             94900   
     lowest_quintile                                  0   
     middle_quintile                              49300   
     percentiles_81_90                            94900   
     percentiles_91_95                           121400   
     percentiles_96_99                           153200   
     second_quintile                              30000   
     top_1_percent                               294300   
1980 fourth_quintile                              66200   
     highest_quintile                             93100   
     lowest_quintile                                  0   
     middle_quintile                              47700   
     percentiles_81_90                            93100   
     percentiles_91_95                           119900   
     percentiles_96_99                           150100   
     second_quintile                              28800   
     top_1_percent                               289300   

household_size          minimum_income_household_size_4  
year variable                                            
1979 fourth_quintile                              78600  
     highest_quintile                            109600  
     lowest_quintile                                  0  
     middle_quintile                              56900  
     percentiles_81_90                           109600  
     percentiles_91_95                           14

In [21]:
pivoted_minimums.reset_index(inplace=True)
pivoted_minimums.head(18)

household_size,year,variable,minimum_income_household_size_1,minimum_income_household_size_2,minimum_income_household_size_3,minimum_income_household_size_4
0,1979,fourth_quintile,39300,55600,68000,78600
1,1979,highest_quintile,54800,77500,94900,109600
2,1979,lowest_quintile,0,0,0,0
3,1979,middle_quintile,28400,40200,49300,56900
4,1979,percentiles_81_90,54800,77500,94900,109600
5,1979,percentiles_91_95,70100,99100,121400,140100
6,1979,percentiles_96_99,88500,125100,153200,176900
7,1979,second_quintile,17300,24500,30000,34700
8,1979,top_1_percent,169900,240300,294300,339800
9,1980,fourth_quintile,38200,54100,66200,76500


In [22]:
pivoted_minimums.columns

Index(['year', 'variable', 'minimum_income_household_size_1',
       'minimum_income_household_size_2', 'minimum_income_household_size_3',
       'minimum_income_household_size_4'],
      dtype='object', name='household_size')

In [23]:
pivoted_minimums.rename(columns={'variable' : 'income_group'}, inplace=True)

In [24]:
all_data = pd.merge(all_data, pivoted_minimums, on=['income_group', 'year'], how='left')

In [25]:
print ("all_data now has ", len(all_data.columns), "columns")
print (all_data.columns)

all_data now has  70 columns
Index(['household_type', 'income_group', 'year', 'num_households',
       'num_children', 'num_adults', 'num_elderly', 'num_people',
       'market_income', 'social_insurance_benefits',
       'inc_before_transfers_taxes', 'means_tested_transfers', 'federal_taxes',
       'inc_after_transfers_taxes', 'wages', 'employee_contrib_deferred_comp',
       'employer_contrib_health_ins', 'employer_share_payroll_taxes',
       'federal_unemployment_tax', 'corp_tax_borne_by_labor',
       'business_income', 'capital_gains', 'tax_exempt_interest',
       'taxable_interest', 'positive_rental_income', 'dividends',
       'corp_tax_borne_by_capital', 'other_market_income', 'social_security',
       'medicare', 'unemployment_insurance', 'workers_compensation',
       'medicaid', 'snap', 'ssi', 'other_transfers', 'individual_income_tax',
       'payroll_taxes', 'corporate_income_tax', 'excise_taxes',
       'means_tested_transfers_rate', 'medicaid_rate', 'snap_rate', 'ssi_

In [26]:
all_data.to_csv('output\distribution_data.csv', index=False)